In [1]:
import torch 
from torch import nn

class ReviewClassifier(nn.Module):
    def __init__(self):
        super().__init__()        
        self.mlp = nn.Sequential(
            nn.Linear(1024, 150),
            nn.ReLU(),
            nn.Linear(150, 15),
            nn.ReLU(),
            nn.Linear(15, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):        
        return self.mlp(x)

In [2]:
import pandas as pd 
df = pd.read_feather("amazon_with_embeddings.bin")

In [3]:
df = df.sample(frac=1.0)
train_df = df[:25000]
val_df = df[25000:30000]
test_df = df[30000:]

In [5]:
from torchmetrics import Accuracy
from tqdm.notebook import tqdm 
import numpy as np 
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()
model = ReviewClassifier()
bce_loss = nn.BCELoss() # TODO: https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html

batch_size = 128
adam_optimizer = torch.optim.Adam(model.parameters(), lr=0.0005) # None #TODO: https://pytorch.org/docs/stable/generated/torch.optim.Adam.html
nr_of_epochs = 20
losses = []
for epoch in tqdm(range(nr_of_epochs)):
    train_df = train_df.sample(frac=1.0)
    current_index = 0 
    while current_index < len(train_df):        
        batch_x = np.array(
            train_df.embeddings[current_index:(current_index + batch_size)].tolist()
        )
        batch_y = np.array(
            train_df.label[current_index: (current_index + batch_size)].tolist()
        )
        
        tensor_batch_x = torch.Tensor(batch_x)
        tensor_batch_y = torch.Tensor(batch_y).reshape(-1, 1)
        
        pred = model.forward(tensor_batch_x)
        loss = bce_loss(pred, tensor_batch_y)
 
        # Backpropagation
        loss.backward() # Liczenie gradientu wag modelu
        adam_optimizer.step() # adam oblicza nowe parametry sieci 
        adam_optimizer.zero_grad()
        
        current_index += batch_size
        losses.append(float(loss.detach().numpy()))
        
    model.eval()
    metric = Accuracy(task='BINARY')
    for _, eval_row in tqdm(val_df.iterrows(), desc='evaluating'):
        pred = model.forward(
            torch.Tensor(np.array(eval_row.embeddings))
        )
        metric.update(pred, torch.Tensor([eval_row.label]))
    metric_value = metric.compute()
    writer.add_scalar('Accuracy/eval', metric_value, epoch)

        

  0%|          | 0/20 [00:00<?, ?it/s]

NameError: name 'np' is not defined

In [29]:
from torchmetrics import Accuracy
metric = Accuracy(task='BINARY')

In [34]:
pred

tensor([0.0046], grad_fn=<SigmoidBackward0>)